In [25]:
import rasterio
from rasterio.windows import Window
import numpy as np
from PIL import Image
import os

In [26]:
def list_tiff_files(directory_path):
    tiff_files = [file for file in os.listdir(directory_path) if file.endswith('.tif') or file.endswith('.tiff')]
    return tiff_files

def list_tiff_files_without_prepost(directory_path):
    tiff_files = [file.replace('_pre_disaster.tif','').replace('_post_disaster.tif','') for file in os.listdir(directory_path) if file.endswith('.tif') or file.endswith('.tiff')]
    return tiff_files

def list_all_city_from_files(directory_path):
    tiff_files = [file.split('_')[0] for file in os.listdir(directory_path) if file.endswith('.tif') or file.endswith('.tiff')]
    city_list = []
    [city_list.append(item) for item in tiff_files if item not in city_list]
    return city_list

In [27]:
import rasterio

def calculate_pixel_dimensions_from_coordinates(raster_path, x_min, y_min, x_max, y_max):
    # Open the raster file
    with rasterio.open(raster_path) as src:
        # Convert coordinates to pixel indices
        row_min, col_min = src.index(x_min, y_max)
        row_max, col_max = src.index(x_max, y_min)
        
        # Calculate width and height in pixels
        width_pixels = col_max - col_min
        height_pixels = row_max - row_min
        
        return width_pixels, height_pixels

# Example usage
raster_path = 'turkey_earthquake/images/reprojected_data/Osmaniye_20221227_10300100DF069700_pre_disaster.tif'
x_min, y_min, x_max, y_max = 36.2199210777483458,37.0467354080691820,36.2736677870815782,37.0997778649585754

width_pixels, height_pixels = calculate_pixel_dimensions_from_coordinates(raster_path, x_min, y_min, x_max, y_max)
print("Width in pixels:", width_pixels)
print("Height in pixels:", height_pixels)

Width in pixels: 17553
Height in pixels: 17323


In [28]:
def get_raster_dimensions(raster_path):
    with rasterio.open(raster_path) as src:
        width = src.width
        height = src.height
        return width, height

In [29]:
get_raster_dimensions(raster_path)

(17553, 17325)

In [178]:
# import rasterio
# from rasterio.windows import Window

# def tile_tiff_to_tiff(pre_disaster_tiff, post_disaster_tiff, tile_size=(1024, 1024), output_folder='tiles'):
#     print("Begin tile:",pre_disaster_tiff.split('/')[-1])
#     # Make sure the output directory exists
#     os.makedirs(output_folder, exist_ok=True)

#     # List to store bounding box coordinates
#     bounding_boxes = []

#     # Open the pre-disaster and post-disaster images
#     with rasterio.open(pre_disaster_tiff) as pre_src, rasterio.open(post_disaster_tiff) as post_src:
#         # Check if both images have the same CRS
#         if pre_src.crs != post_src.crs:
#             raise ValueError("Both images must have the same CRS")

#         # Determine the overlapping region (in the CRS's units)
#         left = max(pre_src.bounds.left, post_src.bounds.left)
#         bottom = max(pre_src.bounds.bottom, post_src.bounds.bottom)
#         right = min(pre_src.bounds.right, post_src.bounds.right)
#         top = min(pre_src.bounds.top, post_src.bounds.top)

#         # Convert coordinates to pixel indices
#         row_min, col_min = pre_src.index(left, top)
#         row_max, col_max = pre_src.index(right, bottom)
#         # Determine the number of tiles in both dimensions
# #         n_tiles_x = int((right - left) // tile_size[0])
# #         n_tiles_y = int((top - bottom) // tile_size[1])
#         n_tiles_x = int((col_max - col_min) // tile_size[0])
#         n_tiles_y = int((row_max - row_min) // tile_size[1])
#         print("number of tiles:", n_tiles_x, "x", n_tiles_y)
        
#         # Loop through the tiles
#         for i in range(n_tiles_y):
#             for j in range(n_tiles_x):
#                 # Compute the bounding box of the tile
#                 tile_left = left + j * tile_size[0]
#                 tile_bottom = bottom + i * tile_size[1]
#                 tile_right = tile_left + tile_size[0]
#                 tile_top = tile_bottom + tile_size[1]
#                 bounding_boxes.append((tile_left, tile_bottom, tile_right, tile_top))

#                 # Define the window for the current tile in both images
#                 pre_window = rasterio.windows.from_bounds(tile_left, tile_bottom, tile_right, tile_top, pre_src.transform)
#                 post_window = rasterio.windows.from_bounds(tile_left, tile_bottom, tile_right, tile_top, post_src.transform)

#                 # Read the tile data from both images
#                 pre_data = pre_src.read(window=pre_window)
#                 post_data = post_src.read(window=post_window)

#                 # Generate filenames for the tile
#                 pre_filename = output_folder+pre_disaster_tiff.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.tif'
#                 post_filename = output_folder+post_disaster_tiff.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.tif'
                
#                 # Create output transform
#                 output_transform = rasterio.transform.from_bounds(tile_left, tile_bottom, tile_right, tile_top, tile_size[0], tile_size[1])

#                 # Save the tiles
#                 with rasterio.open(pre_filename, 'w', driver='GTiff', height=tile_size[1], width=tile_size[0],
#                                    count=pre_src.count, dtype=pre_src.dtypes[0], crs=pre_src.crs,
#                                    transform=output_transform) as pre_dst:
#                     pre_dst.write(pre_data)

#                 with rasterio.open(post_filename, 'w', driver='GTiff', height=tile_size[1], width=tile_size[0],
#                                    count=post_src.count, dtype=post_src.dtypes[0], crs=post_src.crs,
#                                    transform=output_transform) as post_dst:
#                     post_dst.write(post_data)

#         print(f'{n_tiles_x * n_tiles_y} tiles created in {output_folder}')

#     return bounding_boxes

In [177]:
# from shapely.geometry import box
# from rasterio.coords import BoundingBox

# with rasterio.open(pre_disaster_tiff) as pre_src, rasterio.open(post_disaster_tiff) as post_src:
#     pre_src_bound = box(*pre_src.bounds)
#     post_src_bound = box(*post_src.bounds)
#     intersection = pre_src_bound.intersection(post_src_bound)
#     col_off, row_off = pre_src.index(left, top)
#     col_off_end, row_off_end = pre_src.index(right, bottom)
#     width = col_off_end - col_off
#     height = row_off_end - row_off
# print(intersection.bounds)
# print(col_off_end, row_off_end)
# print(width,height)

In [30]:
# # This code almost works except the outputs of pre-disaster and post-disaster tiff should have the same geographic coordinates
# import rasterio
# import os
# from rasterio.windows import Window
# from rasterio.coords import BoundingBox

# def tile_tiff_image(pre_disaster_path, post_disaster_path, output_dir):
#     print("Begin tile:",pre_disaster_path.split('/')[-1])
#     # Ensure output directory exists
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     # Open both images
#     with rasterio.open(pre_disaster_path) as pre_src, rasterio.open(post_disaster_path) as post_src:
#         # Assert same CRS
#         assert pre_src.crs == post_src.crs, "CRS of pre_disaster and post_disaster images don't match."
        
#         # Calculate intersection window in pixel coordinates
#         pre_src_bound = box(*pre_src.bounds)
#         post_src_bound = box(*post_src.bounds)
#         intersection = pre_src_bound.intersection(post_src_bound)
#         if intersection.is_empty:
#             raise ValueError("The two images do not overlap.")

#         left, bottom, right, top = intersection.bounds
#         col_off, row_off = pre_src.index(left, top)
#         col_off_end, row_off_end = pre_src.index(right, bottom)
#         width = col_off_end - col_off
#         height = row_off_end - row_off

#         # Tile size
#         tile_size = 1024

#         # Calculate the number of tiles in x and y direction within the intersection
#         n_tiles_x = int(width / tile_size)
#         n_tiles_y = int(height / tile_size)
#         print("number of tiles:", n_tiles_x, "x", n_tiles_y, "=",n_tiles_x*n_tiles_y)

#         # Tile the intersection region
#         for i in range(n_tiles_x):
#             for j in range(n_tiles_y):
#                 window = Window(col_off + i*tile_size, row_off + j*tile_size, tile_size, tile_size)
                
#                 # Read pre_disaster tile
#                 pre_tile = pre_src.read(window=window)
#                 pre_tile_meta = pre_src.meta.copy()
#                 pre_tile_meta.update({
#                     "height": tile_size,
#                     "width": tile_size,
#                     "transform": rasterio.windows.transform(window, pre_src.transform)
#                 })
#                 pre_output_path = output_dir+pre_disaster_path.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.tif'
#                 with rasterio.open(pre_output_path, "w", **pre_tile_meta) as dest:
#                     dest.write(pre_tile)

#                 # Read post_disaster tile
#                 post_tile = post_src.read(window=window)
#                 post_tile_meta = post_src.meta.copy()
#                 post_tile_meta.update({
#                     "height": tile_size,
#                     "width": tile_size,
#                     "transform": rasterio.windows.transform(window, post_src.transform)
#                 })
#                 post_output_path = output_dir+post_disaster_path.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.tif'
#                 with rasterio.open(post_output_path, "w", **post_tile_meta) as dest:
#                     dest.write(post_tile)

In [32]:
import rasterio
import os
from rasterio.windows import Window
from rasterio.coords import BoundingBox
from shapely.geometry import box

def tile_tiff_image(pre_disaster_path, post_disaster_path, output_dir):
    print("Begin tile:",pre_disaster_path.split('/')[-1])
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open both images
    with rasterio.open(pre_disaster_path) as pre_src, rasterio.open(post_disaster_path) as post_src:
        # Assert same CRS
        assert pre_src.crs == post_src.crs, "CRS of pre_disaster and post_disaster images don't match."
                
        # Calculate intersection window in pixel coordinates
        pre_src_bound = box(*pre_src.bounds)
        post_src_bound = box(*post_src.bounds)
        intersection = pre_src_bound.intersection(post_src_bound)
        if intersection.is_empty:
            raise ValueError("The two images do not overlap.")

        left, bottom, right, top = intersection.bounds
        col_off, row_off = pre_src.index(left, top)
        col_off_end, row_off_end = pre_src.index(right, bottom)
        width = col_off_end - col_off
        height = row_off_end - row_off

        # Tile size
        tile_size = 1024

        # Calculate the number of tiles in x and y direction within the intersection
        n_tiles_x = int(width / tile_size)
        n_tiles_y = int(height / tile_size)
        print("number of tiles:", n_tiles_x, "x", n_tiles_y, "=",n_tiles_x*n_tiles_y)

        # Tile the intersection region
        for i in range(n_tiles_x):
            for j in range(n_tiles_y):
                
                pre_x_start, pre_y_start = pre_src.index(left, top)[1] + i*tile_size, pre_src.index(left, top)[0] + j*tile_size
                post_x_start, post_y_start = post_src.index(left, top)[1] + i*tile_size, post_src.index(left, top)[0] + j*tile_size

                # Window for each raster based on their respective resolutions
                pre_window = Window(pre_x_start, pre_y_start, tile_size, tile_size)
                post_window = Window(post_x_start, post_y_start, tile_size, tile_size)
                
                # Read pre_disaster tile
                try:
                    pre_tile = pre_src.read(window=pre_window)
                    pre_tile_meta = pre_src.meta.copy()
                    pre_tile_meta.update({
                        "height": tile_size,
                        "width": tile_size,
                        "transform": rasterio.windows.transform(pre_window, pre_src.transform)
                    })
                    pre_output_path = output_dir+pre_disaster_path.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.tif'
                    with rasterio.open(pre_output_path, "w", **pre_tile_meta) as dest:
                        dest.write(pre_tile)
                except Exception as exc:
                    print(exc,pre_disaster_path)

                # Read post_disaster tile
                try:
                    post_tile = post_src.read(window=post_window)
                    post_tile_meta = post_src.meta.copy()
                    post_tile_meta.update({
                        "height": tile_size,
                        "width": tile_size,
                        "transform": rasterio.windows.transform(post_window, post_src.transform)
                    })
                    post_output_path = output_dir+post_disaster_path.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.tif'
                    with rasterio.open(post_output_path, "w", **post_tile_meta) as dest:
                        dest.write(post_tile)
                except Exception as exc:
                    print(exc,post_disaster_path)

In [34]:
img_dir = 'turkey_earthquake/images/reprojected_data/'
label_dir = ''

In [35]:
city_list = list_all_city_from_files(img_dir)
city_list

['Osmaniye']

In [36]:
city = 'Osmaniye'
pre_disaster_tiff = [filename for filename in list_tiff_files(img_dir) if city in filename and '_pre_' in filename][0]
pre_disaster_tiff

'Osmaniye_20221227_10300100DF069700_pre_disaster.tif'

In [37]:
%%time
filename = list_tiff_files_without_prepost(img_dir)[0]
city = filename.split('_')[0]
pre_disaster_tiff = img_dir + [filename for filename in list_tiff_files(img_dir) if city in filename and '_pre_' in filename][0]
post_disaster_tiff = img_dir + [filename for filename in list_tiff_files(img_dir) if city in filename and '_post_' in filename][0]
output_folder = 'turkey_earthquake/images/tile_images/'+city+'/'

# output_path = output_folder+pre_disaster_tiff.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.png'
tile_tiff_image(pre_disaster_path = pre_disaster_tiff, post_disaster_path = post_disaster_tiff, output_dir=output_folder)

Begin tile: Osmaniye_20221227_10300100DF069700_pre_disaster.tif
number of tiles: 13 x 17 = 221
CPU times: user 2min 14s, sys: 7.15 s, total: 2min 21s
Wall time: 2min 23s


In [242]:
filename = list_tiff_files_without_prepost(img_dir)[0]
pre_disaster_tiff = img_dir + filename + '_pre_disaster.tif'
post_disaster_tiff = img_dir + filename + '_post_disaster.tif'
output_folder = 'turkey_earthquake/images/tile_images/'
pre_disaster_tiff.split('/')[-1]

'Osmaniye_20221227_10300100DF069700_pre_disaster.tif'

In [283]:
# ##### FOR DEBUG
# filename = list_tiff_files_without_prepost(img_dir)[0]
# city = filename.split('_')[0]
# pre_disaster_tiff = img_dir + [filename for filename in list_tiff_files(img_dir) if city in filename and '_pre_' in filename][0]
# post_disaster_tiff = img_dir + [filename for filename in list_tiff_files(img_dir) if city in filename and '_post_' in filename][0]
# pre_disaster_path = pre_disaster_tiff
# post_disaster_path = post_disaster_tiff
# output_dir=output_folder

# print("Begin tile:",pre_disaster_path.split('/')[-1])
# # Ensure output directory exists
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Open both images
# with rasterio.open(pre_disaster_path) as pre_src, rasterio.open(post_disaster_path) as post_src:
#     # Assert same CRS
#     assert pre_src.crs == post_src.crs, "CRS of pre_disaster and post_disaster images don't match."

#     # Calculate intersection window in pixel coordinates
#     pre_src_bound = box(*pre_src.bounds)
#     post_src_bound = box(*post_src.bounds)
#     intersection = pre_src_bound.intersection(post_src_bound)
#     if intersection.is_empty:
#         raise ValueError("The two images do not overlap.")

#     left, bottom, right, top = intersection.bounds
#     col_off, row_off = pre_src.index(left, top)
#     col_off_end, row_off_end = pre_src.index(right, bottom)
#     width = col_off_end - col_off
#     height = row_off_end - row_off
#     print("Intersection Coordinates:",intersection.bounds)
#     print("Raster Size:",width,height)

#     # Tile size
#     tile_size = 1024

#     # Calculate the number of tiles in x and y direction within the intersection
#     n_tiles_x = int(width / tile_size)
#     n_tiles_y = int(height / tile_size)
#     print("number of tiles:", n_tiles_x, "x", n_tiles_y, "=",n_tiles_x*n_tiles_y)
    
#     i = 0
#     j = 0
#     pre_x_start, pre_y_start = pre_src.index(left, top)[0] + i*tile_size, pre_src.index(left, top)[1] + j*tile_size
#     post_x_start, post_y_start = post_src.index(left, top)[0] + i*tile_size, post_src.index(left, top)[1] + j*tile_size
#     print(left,top)
#     print(pre_x_start,pre_y_start)
#     print(post_x_start,post_y_start)
# #     print(pre_src.index(left, top),post_src.index(left, top))
# #     print(pre_src.index(left, top),post_src.index(left, top))
# #     print(post_src.index(left, top)[0])

Begin tile: Osmaniye_20221227_10300100DF069700_pre_disaster.tif
Intersection Coordinates: (36.219921077748346, 37.05650981186665, 36.27366778708158, 37.099777864958575)
Raster Size: 14130 17553
number of tiles: 13 x 17 = 221
36.219921077748346 37.099777864958575
0 0
2190 10548


In [276]:
post_src.index(37.05666, 36.27349)

(251492, 263004)

In [281]:
intersection.bounds

(36.219921077748346, 37.05650981186665, 36.27366778708158, 37.099777864958575)

In [297]:
post_src.res

(3.3143989590560466e-06, 3.3143989590560466e-06)

In [298]:
pre_src.res

(3.0619671471104993e-06, 3.0619671471104993e-06)

In [280]:
post_src.height

15245

In [288]:
pre_src.index(left, bottom)

(14130, 0)

In [287]:
post_src.index(left, bottom)

(15245, 10548)

### Run Script for every city

In [205]:
from tqdm import tqdm
for city in tqdm(city_list[3:]):
    pre_disaster_tiff = img_dir + [filename for filename in list_tiff_files(img_dir) if city in filename and '_pre_' in filename][0]
    post_disaster_tiff = img_dir + [filename for filename in list_tiff_files(img_dir) if city in filename and '_post_' in filename][0]
    output_folder = 'turkey_earthquake/images/tile_images/'+city+'/'
    tile_tiff_image(pre_disaster_path = pre_disaster_tiff, post_disaster_path = post_disaster_tiff, output_dir=output_folder)

0it [00:00, ?it/s]


### Tiling tiff and json

In [38]:
from shapely.ops import transform
import pyproj
from functools import partial
from shapely import wkt

def get_tif_extent(tif_file):
    with rasterio.open(tif_file) as src:
        bounds = src.bounds  # Returns (left, bottom, right, top) coordinates
        crs = src.crs.to_string()
        extent_tuple = (bounds.left, bounds.bottom, bounds.right, bounds.top)
    
    return bounds, crs

def filter_geodataframe_by_bbox(gdf, bbox):
    # Create a bounding box geometry from the given bbox
    bbox_geom = box(*bbox)
    filtered_gdf = gpd.clip(label_gdf, bbox_geom)
    
    return filtered_gdf

def geojson_to_wkt(gdf):
    """
    Convert a GeoJSON file to a specific WKT format.
    
    Parameters:
    - gdf: Label GeoDataFrame
    
    Returns:
    - Dictionary containing the converted data in the wkt format.
    """
    
#     # Load the GeoJSON file into a GeoDataFrame
#     gdf = gpd.read_file(input_geojson_path)
    
    # Convert the GeoDataFrame to the desired format
    features_list = []
    for _, row in gdf.iterrows():
        feature_data = {
            "properties": {
                "feature_type": row.get("obj_type", "Unknown"),
                "subtype": row.get("damage_gra", "Unknown"),
                "name": row.get("name", "Unknown"),
                "building_type": row.get("info", "Unknown")
            },
            "wkt": row.geometry.wkt
        }
        features_list.append(feature_data)
    
    # Create the final dictionary structure
    output_data = {
        "features": {
            "lng_lat": features_list
        },
        "metadata": {
            "notation": "Building block"
        }
    }
    
    return output_data

def convert_to_pixel_coordinates(input_json, extent, image_width, image_height):
    # Extract bounds from the extent parameter
    minx, miny, maxx, maxy = extent
    
    # Input: WKT in geographic coordinates
    wkt_geom = wkt.loads(input_json["wkt"])
    
    # Transformation function: geographic to pixel coordinates
    def transform_to_pixel(x, y, z=None):
        pixel_x = (x - minx) / (maxx - minx) * image_width
        pixel_y = image_height - (y - miny) / (maxy - miny) * image_height
        return pixel_x, pixel_y
    
    # Create a partial function with the transformation
    transformer = partial(transform_to_pixel)
    
    # Apply the transformation
    pixel_geom = transform(transformer, wkt_geom)
    
    # Return the transformed WKT
    return pixel_geom.wkt

def add_pixel_coordinates(input_json, image_width, image_height):
    output_json = input_json.copy()
    output_json["features"]["xy"] = []
    for i in range(len(input_json['features']['lng_lat'])):
        output_json["features"]["xy"].append(input_json["features"]["lng_lat"][i].copy())
        output_json["features"]["xy"][i]["wkt"] = convert_to_pixel_coordinates(input_json["features"]["lng_lat"][i],provided_extent,image_width,image_height)
    return output_json

In [ ]:
import rasterio
import os
from rasterio.windows import Window
from rasterio.coords import BoundingBox
from shapely.geometry import box

def tile_tiff_and_json(pre_disaster_path, post_disaster_path, output_dir):
    print("Begin tile:",pre_disaster_path.split('/')[-1])
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Open both images
    with rasterio.open(pre_disaster_path) as pre_src, rasterio.open(post_disaster_path) as post_src:
        # Assert same CRS
        assert pre_src.crs == post_src.crs, "CRS of pre_disaster and post_disaster images don't match."
                
        # Calculate intersection window in pixel coordinates
        pre_src_bound = box(*pre_src.bounds)
        post_src_bound = box(*post_src.bounds)
        intersection = pre_src_bound.intersection(post_src_bound)
        if intersection.is_empty:
            raise ValueError("The two images do not overlap.")

        left, bottom, right, top = intersection.bounds
        col_off, row_off = pre_src.index(left, top)
        col_off_end, row_off_end = pre_src.index(right, bottom)
        width = col_off_end - col_off
        height = row_off_end - row_off

        # Tile size
        tile_size = 1024

        # Calculate the number of tiles in x and y direction within the intersection
        n_tiles_x = int(width / tile_size)
        n_tiles_y = int(height / tile_size)
        print("number of tiles:", n_tiles_x, "x", n_tiles_y, "=",n_tiles_x*n_tiles_y)

        # Tile the intersection region
        for i in range(n_tiles_x):
            for j in range(n_tiles_y):
                
                pre_x_start, pre_y_start = pre_src.index(left, top)[1] + i*tile_size, pre_src.index(left, top)[0] + j*tile_size
                post_x_start, post_y_start = post_src.index(left, top)[1] + i*tile_size, post_src.index(left, top)[0] + j*tile_size

                # Window for each raster based on their respective resolutions
                pre_window = Window(pre_x_start, pre_y_start, tile_size, tile_size)
                post_window = Window(post_x_start, post_y_start, tile_size, tile_size)
                
                # Read pre_disaster tile
                pre_tile = pre_src.read(window=pre_window)
                pre_tile_meta = pre_src.meta.copy()
                pre_tile_meta.update({
                    "height": tile_size,
                    "width": tile_size,
                    "transform": rasterio.windows.transform(pre_window, pre_src.transform)
                })
                pre_output_path = output_dir+pre_disaster_path.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.tif'
                with rasterio.open(pre_output_path, "w", **pre_tile_meta) as dest:
                    dest.write(pre_tile)

                # Read post_disaster tile
                post_tile = post_src.read(window=post_window)
                post_tile_meta = post_src.meta.copy()
                post_tile_meta.update({
                    "height": tile_size,
                    "width": tile_size,
                    "transform": rasterio.windows.transform(post_window, post_src.transform)
                })
                post_output_path = output_dir+post_disaster_path.split('/')[-1].replace('.tif','')+'_'+str(i)+'_'+str(j)+'.tif'
                with rasterio.open(post_output_path, "w", **post_tile_meta) as dest:
                    dest.write(post_tile)